In [ ]:
import ollama
import time
from utilities import readtext, getconfig
from mattsollamatools import chunk_text_by_sentences

MODEL_NAME = 'llama3.2'

In [ ]:
embedmodel = getconfig()["embedmodel"]
starttime = time.time()
filename = None
with open('sourcedocs.txt') as f:
  filename = f.read()
if filename is None:
  raise Exception('Filename not found!')
print(filename)
text = readtext(filename)
chunks = chunk_text_by_sentences(source_text=text, sentences_per_chunk=7, overlap=0)

In [ ]:
print(f"with {len(chunks)} chunks")
input_1 = """Você irá gerar uma lista de perguntas e respostas no formato JSON. O formato do arquivo de saída deve conter apenas as variáveis "pergunta" e "resposta". 
Sempre termite a pergunta com um ponto de interrogação. Utilize o formato "{'pergunta':'XXXXX', 'resposta': 'XXXXX'}".
Utilize o seguinte texto como recurso:\n\n"""

result = []
for index, chunk in enumerate(chunks):  
  prompt = f'{input_1}{chunk}'
  stream = ollama.generate(model=MODEL_NAME, prompt=prompt, stream=False, format='json', options={"temperature": 0.8})  
  if stream["response"]:      
    res = stream['response'].strip() + ',\n'
    print(res, end='', flush=True)
    result.append(res)

with open('result.json', 'w') as f:
  f.writelines('[\n')
  f.write('\n'.join(result))
  f.writelines('\n]')
